In [14]:
# create a dummy dataset

import numpy as np
import pandas as pd

def make_dataset(name:str, n:int=10):

    x1 = np.random.randint(0,10,[n,3]) / 10
    x2 = np.random.randint(0,10,[n,3]) / 10

    y = np.sum(x1*x2, axis=1, keepdims=True)

    df = pd.DataFrame(np.concatenate((y, x1, x2), axis=1))

    # first columns is "y"
    df.rename(columns={0:'y'}, inplace=True)

    df.to_csv(name, sep='\t', header=None, index=None)

make_dataset('dummy_train.csv', 10000)
make_dataset('dummy_test.csv', 2000)

In [15]:
import tensorflow as tf
# from tensorflow import keras
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Concatenate, Input, Dense, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.layers import LeakyReLU
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

In [19]:
# wide and deep model

def get_record_defaults(n_features):
    zf = tf.zeros(shape=(1,), dtype=tf.float32)
    zi = tf.zeros(shape=(1,), dtype=tf.float32)
    return [zi] + [zf]*n_features

def parse_batch(tf_string):

    # zero column = y
    # columns from 1 to 3 = x1
    # columns from 4 to 6 = x2

    n_features = 3 + 3 

    data = tf.io.decode_csv(
        tf_string, get_record_defaults(n_features), field_delim='\t')

    labels = data[0]

    x1 = tf.stack(data[1:4], axis=-1)
    x2 = tf.stack(data[4:7], axis=-1)

    return (x1, x2), labels


def get_batched_dataset(file_name, batch_size):
    dataset = tf.data.TextLineDataset([file_name])
    dataset = dataset.batch(batch_size, drop_remainder=False)
    dataset = dataset.map(parse_batch)
    return dataset


def model():

    # first model for x1
    first_input = Input((3, ))
    first_dense = Dense(32, kernel_initializer='he_normal',
                        activation=activation)(first_input)
    first_dense = Dropout(0.5)(first_dense)
    first_dense = Dense(16, kernel_initializer='he_normal',
                        activation=activation)(first_dense)

    # second model for x2
    second_input = Input((3, ))
    second_dense = Dense(32, kernel_initializer='he_normal',
                         activation=activation)(second_input)
    second_dense = Dropout(0.5)(second_dense)
    second_dense = Dense(16, kernel_initializer='he_normal',
                         activation=activation)(second_dense)

    # wide model x1 and x2
    wide_input = Concatenate()(
        [first_input, second_input])
    wide_dense = Dense(32, kernel_initializer='he_normal',
                       activation=activation)(wide_input)
    wide_dense = Dropout(0.5)(wide_dense)
    wide_dense = Dense(16, kernel_initializer='he_normal',
                       activation=activation)(wide_dense)

    # concatenate models
    merged = Concatenate()(
        [first_dense, second_dense, wide_dense])
    output_layer = Dense(32, kernel_initializer='he_normal',
                         activation=activation)(merged)
    output_layer = Dense(1, kernel_initializer='he_normal',
                       activation='relu')(output_layer)

    model = Model(inputs=[first_input, second_input], outputs=output_layer)

    model.compile(loss='mean_squared_error', optimizer=optimizer)

    return model


print('Training ANN...')


def n_rows(file_name):
    counter = 0
    with open(file_name, buffering=2**14) as file:
        for _ in file:
            counter += 1
    return counter


file_name_train = 'dummy_train.csv'
num_rows_train = n_rows(file_name_train)

file_name_test = 'dummy_test.csv'
num_rows_test = n_rows(file_name_test)

batch_size = 64

epochs = 150

optimizer = Adam(clipnorm=1.0)

activation = LeakyReLU(alpha=0.05)

estimator = model()

early_stopping = EarlyStopping(monitor='val_loss',
                               mode='min',
                               verbose=1,
                               patience=15,
                               restore_best_weights=True)

history = estimator.fit(get_batched_dataset(file_name_train, batch_size).repeat(),
                        steps_per_epoch=num_rows_train // batch_size,
                        epochs=epochs,
                        validation_data=get_batched_dataset(
                            file_name_test, batch_size).repeat(),
                        validation_steps=num_rows_test // batch_size,
                        callbacks=[early_stopping],
                        verbose=1)

estimator.save('model', include_optimizer=True)

Training ANN...
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/150
156/156 [==============================] - 13s 73ms/step - loss: 0.2348 - val_loss: 0.0390
Epoch 2/150
156/156 [==============================] - 14s 92ms/step - loss: 0.0710 - val_loss: 0.0351
Epoch 3/150
156/156 [==============================] - 10s 63ms/step - loss: 0.0514 - val_loss: 0.0399
Epoch 4/150
156/156 [==============================] - 10s 62ms/step - loss: 0.0415 - val_loss: 0.0422
Epoch 5/150
156/156 [==============================] - 11s 72ms/step - loss: 0.0362 - val_loss: 0.0436
Epoch 6/150
156/156 [==============================] - 7s 44ms/step - loss: 0.0320 - val_loss: 0.0500
Epoch 7/150
156/156 [==============================] - 11s 71ms/step - loss: 0.0274